# r4 Data Science Exercise

On August 24th, I spoke with Katie Sullivan in regard to a position as a data scientist with r4. I received this data challenge and here are my results and process.

### First
I began by opening the excel file, noted it had 4 sheets, one of which was data and the rest were related to the challenge itself.

### Second
I chose to complete this in Python as it is becoming my language of choice, I find that employers prefer it to R (which I am generally stronger in), the visualizations are simpler to handle, and I can easily share the Jupyter Notbook with you in a state that demonstrates the outcome of my operations

### Third
I imported libraries to handle the data and began a short EDA (exploratory data analysis) to determine an idea of what I was working with and to approach the questions that I am posed in A and B.

### Fourth
Plan:
    * Observe the frequency distribution of the time periods
    * Find the average number of times a user uses the portal
    * Find the portal page that is most and least frequently viewed
    * Indicate the time periods of use and view the frequency distributions for each page
        * Make a plot for each page's distribution, "hued" by week
        * Check the mean for each period for each page and see if it "shifts"

#### Initial EDA and transformations
* Import data by utilizing Pandas 'read' functionality and select the appropriate sheet of the data
* Conduct a quick view with 'head' (transposed to make it easier to read, this is personal preference)
    * Noted the User_ID is some sort of hash, I prefer to categorize by counting numbers as strings because thy aren't meant to be operated upon (e.g. adding social security numbers doesn't make sense, so looking at them as stringed categories is more appropriate)
    * Data is not dirty, but is fairly uninformative
        * I will need to construct all of the relevant information

# Questions

## A - What can you tell me about this data? What is in this data? (5 - 20 min)

## B - Does Portal Page usage shift week-to-week (relative to when an User first used the service)?  (20 - 90 min)

## C - How would you improve Portal Page engagement of Users (relative how a User uses the service)?  (10 - 30 min)

# Answers

## A
* This data is of a longitudinal nature (panel data in econometrics) such that there are n > 1 observations transitioning over time.
* Each observation contains (what appears to be) a hashed (encrypted by some measure) userID, an organization ID that clusters (or groups) the entities, a first use date signifiying the period of initial use of the portal, a use date signifying the period of use for the observation, and a portal page indicating the page used in the observation.


* Data dimension = 134,950 x 5
    * Number of users = n = 2798
    * Number of observations = N = 134,950
* Time period: Days (YY-MM-DD)
    * Number of Time Periods = T = 34
    * First time period = 2015-01-07
    * Last time period = 2015-02-09
    * Day with most uses: 2015-01-13 (8166)
    * Day with least uses: 2015-02-09 (468)
* Number of portal pages: 6
    * Most frequently visited page: Discovery Cube (105489 visits)
    * Least frequently visited page: Bullseye Chart (1183 visits)
    * Most frequently visited page on first use: Discovery Cube (5017 visits)
    * Least frequently visited page on first use: Spider Chart (61 visits)
* Number of groups (Org_ID): 10
    * Most populated group: BLN (695 users)
    * Least populated group: WMT (2 users)
    * Most frequently visits pages: EZ (29318 uses)
    * Least frequently visits pages: WMT (1923 uses)
    

## B

## C

# Working through Question A

In [1]:
# Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()

In [2]:
# Importing
df = pd.read_excel('/Users/mbair/Git/ConvexOptimizationWithPython/Data/DSx.xlsx', sheet_name = 'Data')
df.head(5)

,User_ID,Org_ID,First_Use_Date,Use_Date,Portal_Page
0,07e1e890-947a-42a5-bff6-a6905f6dc2cf-1046-10b2...,P158,2015-01-12,2015-01-12,Discovery Cube
1,07e1e890-947a-42a5-bff6-a6905f6dc2cf-1046-10b2...,P158,2015-01-12,2015-01-12,Discovery Cube
2,07e1e890-947a-42a5-bff6-a6905f6dc2cf-1046-10b2...,P158,2015-01-12,2015-01-12,Discovery Cube
3,07e1e890-947a-42a5-bff6-a6905f6dc2cf-1046-10b2...,P158,2015-01-12,2015-01-12,Discovery Cube
4,07e1e890-947a-42a5-bff6-a6905f6dc2cf-1046-10b2...,P158,2015-01-12,2015-01-12,Discovery Cube


In [3]:
# Control for some naive levels of bad data

# Make sure the data only appears where the fi
df = df[df['First_Use_Date'] <= df['Use_Date']]

# Remove rows with null - the data is fairly uninformative in its current state so replacing null values with anything at this point is not going to help with later analysis
df.dropna(axis=0, how='any', inplace=True)

In [4]:
# Find a few specifics of the data
minDate = df['Use_Date'].min()
maxDate = df['Use_Date'].max()
uniqueDate = df['Use_Date'].nunique()
print('The time span is from {} to {} with {} periods.'.format(minDate, maxDate, uniqueDate))
print()
n = df['User_ID'].nunique()
g = df['Org_ID'].nunique()
avgN = df['User_ID'].value_counts().mean()
print('The number of users is {}, appearing approximately {:,.2f} times, grouped under {} organizations.'.format(n, avgN, g))
print()
pPage = df['Portal_Page'].nunique()
print('There are {} unique portal pages titled: {}.'.format(pPage, list(df['Portal_Page'].unique())))
print()
print(df['Portal_Page'].value_counts())
print()
print(df[df['First_Use_Date'] == df['Use_Date']]['Portal_Page'].value_counts())
print()
print(df.groupby('Org_ID')['User_ID'].nunique())
print()
print(df['Org_ID'].value_counts())
print()
print(df['Use_Date'].value_counts())
print()
df.describe().T

The time span is from 2015-01-07 00:00:00 to 2015-02-09 00:00:00 with 34 periods.

The number of users is 2798, appearing approximately 48.23 times, grouped under 10 organizations.

There are 6 unique portal pages titled: ['Discovery Cube', 'Fingerprint Chart', 'Mashboard', 'Spider Chart', 'Bullseye Chart', 'Product Recommendation'].

Discovery Cube            105489
Fingerprint Chart          21802
Product Recommendation      2711
Mashboard                   2339
Spider Chart                1426
Bullseye Chart              1183
Name: Portal_Page, dtype: int64

Discovery Cube            5017
Fingerprint Chart         1370
Product Recommendation     283
Mashboard                  164
Bullseye Chart              78
Spider Chart                61
Name: Portal_Page, dtype: int64

Org_ID
ASP      58
BLN     695
DGO     618
EZ      242
NW      515
ONG      75
P158    378
SBD       7
USS     208
WMT       2
Name: User_ID, dtype: int64

EZ      29318
P158    28914
NW      20316
BLN     15859
A

,count,unique,top,freq,first,last
User_ID,134950,2798,eb43dc75-8c60-487c-9615-4e750042a0e3-26930-7f4...,1096,NaN,NaN
Org_ID,134950,10,EZ,29318,NaN,NaN
First_Use_Date,134950,7,2015-01-07 00:00:00,25134,2015-01-07 00:00:00,2015-01-13 00:00:00
Use_Date,134950,34,2015-01-13 00:00:00,8166,2015-01-07 00:00:00,2015-02-09 00:00:00
Portal_Page,134950,6,Discovery Cube,105489,NaN,NaN


# Working through Question B

In [5]:
# Add variables for manipulation and for simplicity in viewing

# Use this instead of the hashed User_ID for indexing - just easier for me to view if I need to
df['id'] = pd.Categorical((pd.factorize(df['User_ID'])[0] + 1).astype(str))

df['group'] = df['Org_ID'].astype('category').cat.rename_categories(range(1, df['Org_ID'].nunique() + 1))
df['period'] = df['Use_Date'].astype('category').cat.rename_categories(range(1, df['Use_Date'].nunique() + 1))
df['week'] = df['Use_Date'].dt.week
df['page'] = pd.Categorical((pd.factorize(df['Portal_Page'])[0] + 1).astype(str))

df.head(5)

,User_ID,Org_ID,First_Use_Date,Use_Date,Portal_Page,id,group,period,week,page
0,07e1e890-947a-42a5-bff6-a6905f6dc2cf-1046-10b2...,P158,2015-01-12,2015-01-12,Discovery Cube,1,7,6,3,1
1,07e1e890-947a-42a5-bff6-a6905f6dc2cf-1046-10b2...,P158,2015-01-12,2015-01-12,Discovery Cube,1,7,6,3,1
2,07e1e890-947a-42a5-bff6-a6905f6dc2cf-1046-10b2...,P158,2015-01-12,2015-01-12,Discovery Cube,1,7,6,3,1
3,07e1e890-947a-42a5-bff6-a6905f6dc2cf-1046-10b2...,P158,2015-01-12,2015-01-12,Discovery Cube,1,7,6,3,1
4,07e1e890-947a-42a5-bff6-a6905f6dc2cf-1046-10b2...,P158,2015-01-12,2015-01-12,Discovery Cube,1,7,6,3,1


In [6]:
# Create variables for values

# Total times each unique user appears in the data set
df['totalPageUsesOverall'] = df.groupby('Portal_Page')['Portal_Page'].transform('count')
df['totalPortalUsesByUser'] = df.groupby('User_ID')['User_ID'].transform('count')
df['totalPortalByUserByWeek'] = df.groupby(['User_ID', 'week'])['User_ID'].transform('count')
df['totalPageUsesByUser'] = df.groupby(['User_ID', 'Portal_Page'])['Portal_Page'].transform('count')
df['totalPageByUserByWeek'] = df.groupby(['User_ID', 'week', 'Portal_Page'])['Portal_Page'].transform('count')
df['totalPageUsesByUserByDay'] = df.groupby(['User_ID', 'Use_Date', 'Portal_Page'])['Portal_Page'].transform('count')

df.head()

,User_ID,Org_ID,First_Use_Date,Use_Date,Portal_Page,id,group,period,week,page,totalPageUsesOverall,totalPortalUsesByUser,totalPortalByUserByWeek,totalPageUsesByUser,totalPageByUserByWeek,totalPageUsesByUserByDay
0,07e1e890-947a-42a5-bff6-a6905f6dc2cf-1046-10b2...,P158,2015-01-12,2015-01-12,Discovery Cube,1,7,6,3,1,105489,61,25,55,25,11
1,07e1e890-947a-42a5-bff6-a6905f6dc2cf-1046-10b2...,P158,2015-01-12,2015-01-12,Discovery Cube,1,7,6,3,1,105489,61,25,55,25,11
2,07e1e890-947a-42a5-bff6-a6905f6dc2cf-1046-10b2...,P158,2015-01-12,2015-01-12,Discovery Cube,1,7,6,3,1,105489,61,25,55,25,11
3,07e1e890-947a-42a5-bff6-a6905f6dc2cf-1046-10b2...,P158,2015-01-12,2015-01-12,Discovery Cube,1,7,6,3,1,105489,61,25,55,25,11
4,07e1e890-947a-42a5-bff6-a6905f6dc2cf-1046-10b2...,P158,2015-01-12,2015-01-12,Discovery Cube,1,7,6,3,1,105489,61,25,55,25,11


In [7]:
# Total number of days each unique user appears in the data set
numDays = pd.DataFrame(df.groupby('User_ID')['Use_Date'].nunique())
numDays.columns = ['numDaysByUser']
df = pd.merge(df, numDays, how = 'inner', on = 'User_ID')

# Total number of weeks each unique user appears in the data set
numWeeks = pd.DataFrame(df.groupby('User_ID')['week'].nunique())
numWeeks.columns = ['numWeeksByUser']
df = pd.merge(df, numWeeks, how = 'inner', on = 'User_ID')

df.head()

,User_ID,Org_ID,First_Use_Date,Use_Date,Portal_Page,id,group,period,week,page,totalPageUsesOverall,totalPortalUsesByUser,totalPortalByUserByWeek,totalPageUsesByUser,totalPageByUserByWeek,totalPageUsesByUserByDay,numDaysByUser,numWeeksByUser
0,07e1e890-947a-42a5-bff6-a6905f6dc2cf-1046-10b2...,P158,2015-01-12,2015-01-12,Discovery Cube,1,7,6,3,1,105489,61,25,55,25,11,16,4
1,07e1e890-947a-42a5-bff6-a6905f6dc2cf-1046-10b2...,P158,2015-01-12,2015-01-12,Discovery Cube,1,7,6,3,1,105489,61,25,55,25,11,16,4
2,07e1e890-947a-42a5-bff6-a6905f6dc2cf-1046-10b2...,P158,2015-01-12,2015-01-12,Discovery Cube,1,7,6,3,1,105489,61,25,55,25,11,16,4
3,07e1e890-947a-42a5-bff6-a6905f6dc2cf-1046-10b2...,P158,2015-01-12,2015-01-12,Discovery Cube,1,7,6,3,1,105489,61,25,55,25,11,16,4
4,07e1e890-947a-42a5-bff6-a6905f6dc2cf-1046-10b2...,P158,2015-01-12,2015-01-12,Discovery Cube,1,7,6,3,1,105489,61,25,55,25,11,16,4


In [8]:
# Relevant Averages

# Average daily portal usage for each unique user
df['avgPortalUsesByUser'] = df['totalPortalUsesByUser'] / df['numDaysByUser']
# Average weekly portal usage for each unique user
df['avgPortalUsesByUserByWeek'] = df['totalPortalByUserByWeek'] / df['numWeeksByUser']

# Average page usage for each unique user
df['avgPageOverall'] = df['totalPageUsesByUser'] / df['numDaysByUser']

# Average weekly page usage for each unique user
df['avgPageWeekly'] = df['totalPageByUserByWeek'] / df['numWeeksByUser']

# Like to find the average number of time sa protal page is used on average by week given the total number of times users access a particular page ina aparticular week
df.head()

,User_ID,Org_ID,First_Use_Date,Use_Date,Portal_Page,id,group,period,week,page,...,totalPortalByUserByWeek,totalPageUsesByUser,totalPageByUserByWeek,totalPageUsesByUserByDay,numDaysByUser,numWeeksByUser,avgPortalUsesByUser,avgPortalUsesByUserByWeek,avgPageOverall,avgPageWeekly
0,07e1e890-947a-42a5-bff6-a6905f6dc2cf-1046-10b2...,P158,2015-01-12,2015-01-12,Discovery Cube,1,7,6,3,1,...,25,55,25,11,16,4,3.8125,6.25,3.4375,6.25
1,07e1e890-947a-42a5-bff6-a6905f6dc2cf-1046-10b2...,P158,2015-01-12,2015-01-12,Discovery Cube,1,7,6,3,1,...,25,55,25,11,16,4,3.8125,6.25,3.4375,6.25
2,07e1e890-947a-42a5-bff6-a6905f6dc2cf-1046-10b2...,P158,2015-01-12,2015-01-12,Discovery Cube,1,7,6,3,1,...,25,55,25,11,16,4,3.8125,6.25,3.4375,6.25
3,07e1e890-947a-42a5-bff6-a6905f6dc2cf-1046-10b2...,P158,2015-01-12,2015-01-12,Discovery Cube,1,7,6,3,1,...,25,55,25,11,16,4,3.8125,6.25,3.4375,6.25
4,07e1e890-947a-42a5-bff6-a6905f6dc2cf-1046-10b2...,P158,2015-01-12,2015-01-12,Discovery Cube,1,7,6,3,1,...,25,55,25,11,16,4,3.8125,6.25,3.4375,6.25


In [10]:
df.groupby('week')['avgPageWeekly'].values()

AttributeError: Cannot access attribute 'values' of 'SeriesGroupBy' objects, try using the 'apply' method